In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks')

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader

import torchvision
from torchvision import datasets as D
from torchvision import transforms as T

from utils import train_step, test_step
from utils import get_cifar10_dataset, make_dataloader
from utils import simulate_scheduler
from utils import check_install_module
check_install_module("plotly")
check_install_module("jupyterplot")

from plotly import express as px
from plotly import graph_objects as go
from plotly.subplots import make_subplots

from jupyterplot import ProgressPlot

# 지금까지 배운 내용을 조합해 자신의 모델 훈련해보기

### Model
- Convolution layer 개수: input / output feature의 크기가 같도록 하면 늘릴 수 있음
- Convolution layer의 out_channels 조절해보기
- Fully connected layer(nn.Linear)의 features 조절해보기 / 개수 조절해보기

In [ ]:
class MyCNN(nn.Module):
    def __init__(self):
        super().__init__()
        ##### layers here ####
    
    def forward(self, x):
        #### convolutions ####
        
        ######################
        x = x.flatten(1)
        ####      FCs     ####
        
        ######################
        return x

print(MyCNN())

## 예시
예시를 확인하고 자신의 네트워크를 만든 후에는 아래 셀을 지워주세요

In [ ]:
class MyCNN(nn.Module):
    def __init__(self):
        super().__init__()
        ##### layers here ####
        self.conv1 = nn.Conv2d(
            in_channels=3, out_channels=32,
            kernel_size=5, stride=1, padding=2
        )
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1_1 = nn.Conv2d(32, 32, 3, 2, 1)
        self.relu1_1 = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv2d(32, 64, 3, 1, 1)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv2_1 = nn.Conv2d(64, 64, 3, 2, 1)
        self.relu2_1 = nn.ReLU(inplace=True)

        self.conv3 = nn.Conv2d(64, 128, 3, 1, 1)
        self.relu3 = nn.ReLU(inplace=True)
        self.conv3_1 = nn.Conv2d(128, 128, 3, 2, 1)
        self.relu3_1 = nn.ReLU(inplace=True)
        
        self.fc1 = nn.Linear(in_features=128 * 4 * 4, out_features=64)
        self.fc2 = nn.Linear(64, 10)
        
        # 파라미터 초기화 예시
        for module in self.modules():
            if isinstance(module, nn.Conv2d):
                torch.nn.init.kaiming_normal_(module.weight, nonlinearity='relu')
                # torch.nn.init.zeros_(module.bias)
    
    def forward(self, x):
        #### convolutions ####
        x = self.relu1(self.conv1(x))
        x = self.relu1_1(self.conv1_1(x))

        x = self.relu2(self.conv2(x))
        x = self.relu2_1(self.conv2_1(x))

        x = self.relu3(self.conv3(x))
        x = self.relu3_1(self.conv3_1(x))
        
        ######################
        x = x.flatten(1)
        ####      FCs     ####
        x = self.fc1(x)
        x = self.fc2(x)
        ######################
        return x

print(MyCNN())

### Hyperparameters

- Epoch 수
- learning rate
- (optional) batch_size

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
momentum = 0.9
phases = ['train', 'test']

num_epochs = 1
learning_rate = 0.01
batch_size = 64

### Data Augmentation

In [ ]:
data_augmentation = False

dataset = get_cifar10_dataset(random_crop=data_augmentation)
loader = make_dataloader(dataset, batch_size)

# Learning rate scheduling
- gamma: 한 epoch당 얼만큼 줄일 것인가. 숫자가 낮을수록 빠르게 줄어듦

In [ ]:
gamma = 0.9

data = {
    'Learning rate': simulate_scheduler(gamma, num_epochs),
    'Epoch': list(range(num_epochs))
}

px.line(data, x='Epoch', y='Learning rate', width=600, height=400)

In [ ]:
model = MyCNN().to(device)
optimizer = torch.optim.SGD(model.parameters(), learning_rate, 0.9)
# optimizer = torch.optim.Adam(model.parameters(), 0.01) # Adam optimizer 사용입니다. learning rate는 0.01 사용 권장.
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma)

pp = ProgressPlot(
    plot_names=phases,
    line_names=['loss', 'accuracy'],
    x_lim=[0, None],
    x_label='Iteration',
    y_lim=[[0, None], [0, 100]]
)
logs = dict(loss=[], accuracy=[])

accuracy = 0
for epoch in range(num_epochs):
    for inputs, target in loader['train']:
        loss = train_step(model, inputs, target, optimizer, criterion, device)
        pp.update([[loss, -1], [-500, accuracy]])
        logs['loss'].append(loss)
    
    corrects = 0
    for inputs, target in loader['test']:
        output, _ = test_step(model, inputs, target, device=device)
        corrects += (output.argmax(1).cpu() == target).sum().item()
    accuracy = corrects / len(dataset['test']) * 100
    logs['accuracy'].append(accuracy)
    
    print(f'Epoch: {epoch+1} accuracy {accuracy:.2f}')
pp.finalize()

In [ ]:
iteration = list(range(num_epochs * len(loader['train'])))
ext_epoch = [iep+len(loader['train']) for iep in range(0, len(iteration), len(loader['train']))]

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=iteration, y=logs['loss'], name='loss'),
    secondary_y=False
)
fig.add_trace(
    go.Scatter(x=ext_epoch, y=logs['accuracy'], name='Accuracy'),
    secondary_y=True
)

fig.update_xaxes(title_text='Iteration')
fig.update_yaxes(title_text='Loss', secondary_y=False) #, range=[0, 3]) range를 통해 그래프 y축의 범위를 조절
fig.update_yaxes(title_text='Accuracy', secondary_y=True, range=[0,100])
fig.show()